# Investigation of Similarity measures
### Import precomp libs


In [1]:
#import the spacy english model
# "python -m spacy download en"   to download english models
import spacy
nlp = spacy.load('en') # this should take some time like 10s to load
import numpy as np
import scipy as sp
import pandas as pd
import time

In [2]:
from datetime import datetime

In [2]:
# PROJECT structure: data directory with the two csvs of train,test and the ipython notbook in its own directory
# loading the data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
print train.shape, test.shape

(404290, 6) (2345796, 3)


In [3]:
train.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [4]:
#this procedure will be mapped to each row
def eval_sysim(row):
    q1_txt = nlp(unicode(row['question1']))
    q2_txt = nlp(unicode(row['question2']))
    # feature: similarity measure from built-in spacy
    #word net similarity? or hamming distance of the strings?
    sy_sim = q1_txt.similarity(q2_txt)

    #feature: 
    return sy_sim

def u_eval_test(fun):
    #unit test(s)
    utest = train.loc[1]
    print fun(utest)
    utest = train.loc[3]
    print fun(utest)
    utest = train.loc[9]
    print fun(utest)
    


def eval_sysim1(row):
    q1_txt = nlp(unicode(row['question1']))
    q2_txt = nlp(unicode(row['question2']))
    q1 = []
    q2 = []
    for w1 in q1_txt:
        if (w1.is_stop == False):
            q1.append(w1)
    for w2 in q2_txt:
        if (w2.is_stop == False):
            q2.append(w1)
    # feature: similarity measure from built-in spacy
    #word net similarity? or hamming distance of the strings?
    q1 = nlp(q1)
    q2 = nlp(q2)
    sim = 0.0
    
    reduce(lambda x,y: x+y, )
    sy_sim = q1.similarity(q2)

    #feature: 
    return sy_sim

u_eval_test(eval_sysim)
#u_eval_test(eval_sysim1)
    

0.934445645664
0.740894391357
0.819441244603


In [5]:
def jacc_sim(row):
    q1_txt = nlp(unicode(row['question1']))
    q2_txt = nlp(unicode(row['question2']))
    #test jacc sim
    q1 = set()
    q2 = set()
    for w in q1_txt:
        if w.pos_ in ['NOUN','PROPN','VERB']:
            q1.add((w.text, w.pos_))
    for w in q2_txt:
        if w.pos_ in ['NOUN','PROPN','VERB']:
            q2.add((w.text, w.pos_))
    return len(q1.intersection(q2)) / (1.0*len(q1.union(q2)))   

print jacc_sim(train.loc[2])

0.181818181818


In [7]:
from nltk.corpus import wordnet as wn
#IMPORTANCE OF ENTITIES NOT IN BOTH
diff_ents = []
def diff_ents(row):
    q1_txt = nlp(unicode(row['question1']))
    q2_txt = nlp(unicode(row['question2']))
    #test jacc sim
    q1 = set()
    q2 = set()
    q1_n = []
    q2_n = []
    for w in q1_txt.ents:
        q1.add(w)
    for w in q2_txt.ents:
        q2.add(w)
    for n in q1_txt.noun_chunks:
        q1_n.append(n)
    for n in q2_txt.noun_chunks:
        q2_n.append(n)    
    return (q1,q2,q1_n,q2_n) 

a,b,c,d = diff_ents(train.loc[5])
c = map(lambda x: x.text.split(), c)
c = [item for sublist in c for item in sublist]
d = map(lambda x: x.text.split(), d)
d = [item for sublist in d for item in sublist]
rez = []
for word1 in c:
    for word2 in d:
        wordFromList1 = wn.synsets(word1)
        wordFromList2 = wn.synsets(word2)
        if wordFromList1 and wordFromList2: #Thanks to @alexis' note
            s = wordFromList1[0].wup_similarity(wordFromList2[0])
            rez.append(s)
reduce(lambda x,y: x+y, rez)/len(rez)

0.3205414059688226

here we see the pitfall in relying on the similarity measure of the two doccuments as their scope differs(invest in shares vs invest in shares of **INDIA**). To extend this baseline model we will look at parts of speech and entity recognition to help parse scope.

We can however use the similarity as the first filter to remove completely unrelated questions


In [8]:
from nltk.corpus import wordnet as wn

def eval_wordnet(row):
    q1_txt = row['question1'].split()
    q2_txt = row['question2'].split()
    
    #ADD TOKENIZATION
    
    rez=[]
    
    #this is word against every other word
    for word1 in q1_txt:
        for word2 in q2_txt:
            wordFromList1 = wn.synsets(word1)
            wordFromList2 = wn.synsets(word2)
            if wordFromList1 and wordFromList2: #Thanks to @alexis' note
                s = wordFromList1[0].wup_similarity(wordFromList2[0])
                rez.append(s)
    return reduce(lambda x,y: x+y, rez)/len(rez)

utest = train.loc[9]
print eval_wordnet(utest)

utest = train.loc[7]
print eval_wordnet(utest)
  

0.367406582388
0.388970049558


In [9]:
from nltk.corpus import stopwords
objts = []

def parse_clean(row):
    q1_txt = [word for word in row['question1'].split() if word not in stopwords.words('english')]
    q2_txt = [word for word in row['question2'].split() if word not in stopwords.words('english')]
    return (q1_txt,q2_txt)

from nltk.tokenize import word_tokenize
def tokenize(sent):
    tokens = word_tokenize(sent)
    return tokens

def parse_clean(row):
    q1_txt = [word for word in row['question1'].split() if word not in stopwords.words('english')]
    q2_txt = [word for word in row['question2'].split() if word not in stopwords.words('english')]
    return (q1_txt,q2_txt)

import nltk
def pos_tag(tokens):
    return nltk.pos_tag(tokens)


from nltk.stem.porter import PorterStemmer
def stem(tokens):
    pstem = PorterStemmer()
    return map(lambda x: pstem.stem(x),tokens)


from nltk.stem.wordnet import WordNetLemmatizer
def word_stem(tokens):
    rez = []
    for tok in tokens:
        tokdk = nlp(unicode(tok))
        pos = ""
        for x in tokdk:
            pos = x.pos_
        pstem = WordNetLemmatizer()
        print pos
        rez.append(pstem.lemmatize(tok))
    return rez 



I will investigate regression on the POS tagging and have one feature be lemmatized verb similarity and entity similarites with penalty for entities not encluded

In [12]:
# append all text of the questions together for training
texts = map(lambda x:  str(x), train.loc[:,'question1']) \
+ map(lambda x:  str(x), train.loc[:,'question2']) \
+ map(lambda x: str(x), test.loc[:,'question1']) \
+ map(lambda x: str(x), test.loc[:,'question2'])

# delete duplicates
qs = set(texts)


In [33]:
qs = [x for x in qs if x is not None]

lemmas = []
for idx, doc in enumerate(docs):
    lemma = []
    for word in doc:
        if not word.is_punct: lemma.append(word.lemma_)
    lemmas.append(lemma)
    if idx == 20: break

In [14]:
from gensim.models import word2vec
# PRE-PROCESS THE SENTENCES AND STEM / LEMMATIZE!!!!
def pipe_clean(sent):
    
sentz = [pipe_clean(x) for x in qs]
#train our word to vec model and get ready
model = word2vec.Word2Vec(sentz, min_count=1)
model.save("saved_vec_mod.kf")

In [16]:
from gensim.models import word2vec

model = word2vec.Word2Vec.load("saved_vec_mod.kf")

def vec_diff(doc1,doc2):
    a = doc1.vector
    b = doc2.vector
    return a - b
    
def wmdist(sent1,sent2):
    """This requires that the parameters @sent1 @sent2 are lists of strings ideally
    lemmatized and cleaned sentences from above"""
    return model.wmdistance(sent1, sent2)
  

In [32]:
vec_diff(nlp(unicode(list(qs)[2])), nlp(unicode(list(qs)[4])))

(300,)

In [43]:
wmdist(texts[7],texts[train.shape[0]+7])

2.174199494647931

In [55]:
lemmas = []
for idx, q in enumerate(docs):
    lemma = []
    doc = nlp(q.decode('utf-8'))
    for word in doc:
        if not word.is_punct:
            if word.lemma_ == '-PRON-':
                lemma.append(unicode(word))
            else:
                lemma.append(word.lemma_)
    lemmas.append(lemma)
print len(lemmas)


KeyboardInterrupt: 

READ THE DATA and then train model!

In [3]:
test_loc = 'test_dump.bin'
train_loc = 'train_dump.bin'

from spacy.tokens.doc import Doc
test_docs = []
train_docs = []
i = 0
#print datetime.now().strftime('%Y-%m-%d %H:%M:%S')
with open(train_loc, 'rb') as file_:
    for byte_string in Doc.read_bytes(file_):
        #if i%10000 == 0: print i, datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        train_docs.append(Doc(nlp.vocab).from_bytes(byte_string))
        i += 1

In [4]:
i=0
with open(test_loc, 'rb') as file_:
    for byte_string in Doc.read_bytes(file_):
        if i%100000 == 0: print i, datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        test_docs.append(Doc(nlp.vocab).from_bytes(byte_string))
        i += 1
        
#         if i == 10: break
#print datetime.now().strftime('%Y-%m-%d %H:%M:%S')

0 2017-05-13 20:15:25
100000 2017-05-13 20:15:35
200000 2017-05-13 20:15:44
300000 2017-05-13 20:15:54
400000 2017-05-13 20:16:03
500000 2017-05-13 20:16:12
600000 2017-05-13 20:16:22
700000 2017-05-13 20:16:31
800000 2017-05-13 20:16:41
900000 2017-05-13 20:16:52
1000000 2017-05-13 20:17:01
1100000 2017-05-13 20:17:10
1200000 2017-05-13 20:17:19
1300000 2017-05-13 20:17:38
1400000 2017-05-13 20:17:48
1500000 2017-05-13 20:17:57
1600000 2017-05-13 20:18:06
1700000 2017-05-13 20:18:15
1800000 2017-05-13 20:18:24
1900000 2017-05-13 20:18:54
2000000 2017-05-13 20:19:03
2100000 2017-05-13 20:19:12
2200000 2017-05-13 20:19:22
2300000 2017-05-13 20:19:31
2400000 2017-05-13 20:19:40
2500000 2017-05-13 20:20:18
2600000 2017-05-13 20:20:27
2700000 2017-05-13 20:20:36
2800000 2017-05-13 20:20:45
2900000 2017-05-13 20:20:54
3000000 2017-05-13 20:21:04
3100000 2017-05-13 20:21:12
3200000 2017-05-13 20:21:22
3300000 2017-05-13 20:21:32
3400000 2017-05-13 20:22:24
3500000 2017-05-13 20:22:36
3600000

In [5]:
docs = train_docs + test_docs


In [ ]:
lemmas = []
st_tm = time.time()
for idx, doc in enumerate(docs):
    lemma = []
    for word in doc:
        if not word.is_punct:
            if word.lemma_ == '-PRON-':
                lemma.append(unicode(word))
            else:
                lemma.append(word.lemma_)
    lemmas.append(lemma)
    if idx % 100000 == 0:
        print time.time() - st_tm
print lemmas[:3]

0.000380992889404
12.0580840111
18.9055030346
84.5726351738
94.4663629532
103.188327074
110.556419134
116.528983116
205.783671141
217.097140074
233.241887093
275.003391027
288.963165998
303.790642977
407.842146158
429.021991968
442.510020971
453.314794064
462.367565155
468.373139143
480.398862123
613.231790066
641.839128017
653.091371059
660.687510967
666.921568155
686.481702089
697.070707083
723.861688137
732.505973101
739.9001441
1002.07970905
1015.29264998
1033.14704204
1045.67611814
1059.63146114
1091.12354517
1139.30702496
1164.7583971
1175.28036094
1183.44532204
1193.69142008


In [ ]:
print "a"